In [ ]:
def Coupling():
    input_layer = layers.Input(shape = 2)

    s_layer_1 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(input_layer)
    s_layer_2 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(s_layer_1)
    s_layer_3 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(s_layer_2)
    s_layer_4 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(s_layer_3)
    s_layer_5 = layers.Dense(2, activation = "tanh", kernel_regularizer = regularizers.l2(0.01))(s_layer_4)

    s_layer_1 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(input_layer)
    t_layer_2 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(t_layer_1)
    t_layer_3 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(t_layer_2)
    t_layer_4 = layers.Dense(256, activation = "relu", kernel_regularizer = regularizers.l2(0.01))(t_layer_3)
    t_layer_5 = layers.Dense(2, activation = "tanh", kernel_regularizer = regularizers.l2(0.01))(t_layer_4)

    return models.Model(inputs = input_layer, outputs = [s_layer_5, t_layer_5])

In [ ]:
class RealNVP(models.Model):
    def __init__(self, input_dim, coupling_layers, coupling_dim, regularization):
        super(RealNVP, self).__init__()
        self.coupling_layers = coupling_layers
        self.distribution = tfp.distributions.MultivariateNormalDiag(
            loc = [0.0, 0.0], scale_diag = [1.0, 1.0]
        )
        self.masks = np.array(
            [[0, 1], [1, 0]] * (coupling_layers // 2), dtype = "float32"
        )
        self.loss_tracker = metrics.Mean(name = "loss")
        self.layers-list = [
            Coupling(input_dim, coupling_dim, regularization)
            for i in range(coupling_layers)
        ]

        @property
        def metrics(self):
            return [self.loss_tracker]
        def call(self, x, training = True):
            log_det_inv = 0
            direction = 1
            if training:
                direction = -1
            for i in range(self.coupling_layers)[::direction]:
                x_masked = x * self.masks[i]
                reversed_mask = 1 - self.masks[i]
                s, t = self.layers_list[i](x_masked)
                s *= reversed_mask
                t *= reversed_mask
                gate = (direction - 1) / 2
                x = (
                    reversed_mask
                    * (x * tf.exp(direction * s) + direction * t * tf.exp(gate * s))
                    + x_masked
                )
                log_det_inv += gate * tf.reduce_sum(s, axis = 1)
            return x, log_det_inv
            
        def log_loss(self, x):
            y, logdet = self(x)
            log_likelihood = self.distribution.log_prob(y) + logdet
            return -tf.reduce_mean(log_likelihood)
        
        def train_step(self, data):
            with tf.GradientTape() as tape:
                loss = self.log_loss(data)
            g = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            self.loss_tracker.update_state(loss)
            return {"loss": self.loss_tracker.result()}
        
model = RealNVP(
    input_dim = 2,
    coupling_layers = 6,
    coupling_dim = 256,
    retularization = 0.01
)

model.compile(optimizer = optimizers.Adam(learning_rate = 0.0001))

model.fit(
    normalized_data,
    batch_size = 256,
    epochs = 300
)